
# 0902 - Initial Probability Model Kickoff (df2/df3 small runs)

This notebook stages the smaller validation extracts (`df2_valid_small.csv`, `df3_valid_small.csv`) for the Poisson bathtub workflow described in [docs/initial_probability_models.md](docs/initial_probability_models.md). It demonstrates how the refreshed library helpers guide analysts from raw beam counters to fluence-binned failure rates.



## Objectives

* Load the validation subsets and beam monitor data using the documented helper APIs.
* Gate failure events by beam-on exposure and compute fluence-equivalent time with `radbin.core.compute_scaled_time_clipped`.
* Build first-pass fluence bins with `radbin.core.build_and_summarize`, store the summary, and visualise the resulting sigma_SEE profile.
* Run `radbin.glm.poisson_trend_test_plus` as the near-term SMART goal to identify any monotonic drift in the failure rate.


In [ ]:

from __future__ import annotations

from pathlib import Path

import pandas as pd
from IPython.display import display

from lib.beam import read_beam_data, beam_pipeline
from lib.cpld_events import detect_bit_increments
from radbin.core import (
    compute_scaled_time_clipped,
    inspect_scaled_time,
    build_and_summarize,
    plot_cumulative_fails,
    to_datetime_smart,
)
from radbin.glm import poisson_trend_test_plus, format_trend_report
from radbin.plots import bar_rates, plot_scaling_ratio

import matplotlib.pyplot as plt


In [ ]:

DATA_DIR = Path('1_data')
FAILURE_FILES = [
    DATA_DIR / 'df2_valid_small.csv',
    DATA_DIR / 'df3_valid_small.csv',
]
# Adjust if the beam extract uses a different filename locally.
BEAM_FILE = DATA_DIR / 'beam3.csv'

RESULTS_DIR = Path('results') / 'radbin'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV = RESULTS_DIR / 'run_df2_df3_fluence.csv'

print(f"Beam file: {BEAM_FILE}")
print('Failure logs:')
for path in FAILURE_FILES:
    print(f'  - {path}')
print(f"Fluence-bin output -> {OUTPUT_CSV}")



## 1. Load and inspect the beam monitor

The `lib.beam.read_beam_data` helper enforces the column naming conventions adopted in the historical notebooks (`time`, `TID`, `HEH`, `N1MeV`) and flags any non-monotonic counter segments before we proceed.


In [ ]:

beam_df = read_beam_data(BEAM_FILE, run_id=32, plot=True, title='Beam monitor check - df2/df3 subset')
print(f"Loaded {len(beam_df):,} beam rows spanning {beam_df['time'].min()} -> {beam_df['time'].max()}")
beam_df.head()



### Differential dose rates

`lib.beam.beam_pipeline` computes per-sample dose rates and a boolean `beam_on` flag (defaults to `TID_dose_rate > 1e-7 Gy/s`). This mirrors the staging performed in the Poisson bathtub notebooks before bin construction.


In [ ]:

beam_processed = beam_pipeline(beam_df, epsilon=1e-7, debug=True)
beam_processed.head()



### Fluence-equivalent timebase

The probability models use fluence-scaled exposure (`dt_eq`, `t_eq`) rather than wall-clock durations. `radbin.core.compute_scaled_time_clipped` reproduces the notebook logic, including beam-off freezing and adaptive floors. The diagnostic call highlights any extreme scaling ratios.


In [ ]:

beam_eq = compute_scaled_time_clipped(
    beam_processed,
    flux_col='HEH_dose_rate',
    beam_on_col='beam_on',
    mode='fluence',
)
inspect_scaled_time(beam_eq, label='df2/df3 fluence track')
fluence_total = beam_eq.loc[beam_eq['beam_on'] == 1, 'dt_eq'].sum()
print(f'Total integrated fluence (beam-on): {fluence_total:,.3e}')
beam_eq[['time', 'dt', 'dt_eq', 't_eq', 'beam_on']].head()



## 2. Load the CPLD failure logs

The helper below normalises each CSV so that downstream functions receive a monotonic cumulative counter (`failsP_acum`). It auto-detects timestamp columns and, when needed, rebuilds the cumulative sequence from per-event increments or per-bit counters.


In [ ]:

def load_failure_csv(path: Path) -> pd.DataFrame:
    """Return a tidy failure counter table ready for RadBIN utilities."""
    df_raw = pd.read_csv(path)
    candidate_time_cols = [col for col in ['time', 'timestamp', 'event_time', 't', 'datetime'] if col in df_raw.columns]
    if not candidate_time_cols:
        raise KeyError(f'No timestamp column found in {path}. Available columns: {df_raw.columns.tolist()}')
    time_col = candidate_time_cols[0]
    df_raw['time'] = to_datetime_smart(df_raw[time_col])
    df = df_raw.dropna(subset=['time']).sort_values('time').reset_index(drop=True)

    if 'failsP_acum' in df.columns:
        df['failsP_acum'] = pd.to_numeric(df['failsP_acum'], errors='coerce').ffill().fillna(0).astype(int)
    elif 'total_fails' in df.columns:
        df['failsP_acum'] = pd.to_numeric(df['total_fails'], errors='coerce').ffill().fillna(0).astype(int)
    elif 'increment' in df.columns:
        increments = pd.to_numeric(df['increment'], errors='coerce').fillna(0).astype(int)
        df['failsP_acum'] = increments.cumsum().astype(int)
    else:
        bit_cols = [c for c in df.columns if c.startswith('bitn') and c[len('bitn'):].isdigit()]
        if bit_cols:
            counters = df[['time'] + bit_cols].copy()
            events = detect_bit_increments(counters, time_col='time')
            if events.empty:
                df['failsP_acum'] = 0
            else:
                increments = events.groupby('time')['increment'].sum().astype(int)
                df = df.merge(increments.rename('increment'), on='time', how='left')
                df['increment'] = df['increment'].fillna(0).astype(int)
                df['failsP_acum'] = df['increment'].cumsum().astype(int)
        else:
            raise KeyError(f'Could not infer cumulative failures for {path}.')

    if 'increment' not in df.columns:
        df['increment'] = df['failsP_acum'].diff().fillna(df['failsP_acum']).clip(lower=0).astype(int)

    df = (
        df.groupby('time', as_index=False)
          .agg(failsP_acum=('failsP_acum', 'max'), increment=('increment', 'sum'))
          .sort_values('time')
          .reset_index(drop=True)
    )
    df['failsP_acum'] = df['failsP_acum'].cummax().astype(int)
    df['increment'] = df['increment'].clip(lower=0).astype(int)
    return df


In [ ]:

failure_tables = []
for path in FAILURE_FILES:
    if not path.exists():
        raise FileNotFoundError(f'Expected failure log {path} was not found. Update FAILURE_FILES if necessary.')
    table = load_failure_csv(path)
    print(f"{path.name}: {len(table):,} rows, {table['failsP_acum'].iloc[-1]:,} cumulative fails")
    failure_tables.append(table)

failures = pd.concat(failure_tables, ignore_index=True).sort_values('time').reset_index(drop=True)
failures['failsP_acum'] = failures['failsP_acum'].cummax().astype(int)
failures['increment'] = failures['failsP_acum'].diff().fillna(failures['failsP_acum']).clip(lower=0).astype(int)
print(f"Combined: {len(failures):,} rows / {failures['failsP_acum'].iloc[-1]:,} total events")
failures.head()



### Quick visual check

Plotting the cumulative counter ensures we inherited the expected bathtub outline before merging with the beam stream.


In [ ]:
plot_cumulative_fails(failures, title='Cumulative CPLD failures - df2/df3 subset')


## 3. Gate failures by beam exposure

We align the failure timestamps with the processed beam table to preserve only the events observed while the beam was active. The `merge_asof` call mirrors the historical notebooks and attaches the instantaneous dose rate for additional QA.


In [ ]:

aligned = pd.merge_asof(
    failures.sort_values('time'),
    beam_processed[['time', 'beam_on', 'HEH_dose_rate', 'dt']],
    on='time',
    direction='backward'
)
aligned['beam_on'] = aligned['beam_on'].fillna(False)
failures_on = aligned[aligned['beam_on']].copy()
print(f"Beam-gated failures: {len(failures_on):,} / {len(failures):,} rows remain")
failures_on.head()



## 4. Fluence binning and sigma_SEE profile

We reuse the binning strategy from the 0829 notebook: fluence-equal bins, Garwood confidence intervals, minimum exposure and event guardrails, and the standard `(area_run=32, area_ref=1)` normalisation.


In [ ]:

N_BINS = 12
T_min = 0.2 * fluence_total / N_BINS if N_BINS > 0 else None
if T_min:
    print('Target bins: {}, min exposure per bin: {:.3e}'.format(N_BINS, T_min))
else:
    print(f'Target bins: {N_BINS}, min exposure per bin: not set')

bin_summary = build_and_summarize(
    df_beam=beam_processed,
    fails_df=failures_on,
    bin_mode='fluence',
    n_bins=N_BINS,
    flux_col='HEH_dose_rate',
    area_norm=(32, 1),
    alpha=0.35,
    min_events_per_bin=5,
    min_exposure_per_bin=T_min,
)
if bin_summary.empty:
    raise RuntimeError('No fluence bins were produced; check the beam gating and cumulative counters.')

bin_summary['sigma_SEE'] = bin_summary['N'] / bin_summary['T']
print('Binning produced {} bins with median sigma_SEE = {:.3e}'.format(len(bin_summary), bin_summary['sigma_SEE'].median()))
bin_summary.head()


In [ ]:
bin_summary.to_csv(OUTPUT_CSV, index=False)
print(f'Saved fluence summary to {OUTPUT_CSV}')


### Visual diagnostics

* `bar_rates`: log-scale sigma_SEE bars with Garwood error bars.
* `plot_scaling_ratio`: sanity-check the exposure scaling used by `compute_scaled_time_clipped`.


In [ ]:

ax = bar_rates(
    bin_summary,
    y_var='rate',
    title='sigma_SEE per fluence bin - df2/df3 subset',
    logy=True,
)
_ = ax.figure


In [ ]:

ax = plot_scaling_ratio(beam_eq, label='df2/df3 fluence scaling')
_ = ax.figure



## 5. Poisson trend test

With the bins validated, we execute the SMART-goal GLM to quantify monotonic drift. The robust standard errors handle mild over-dispersion, and the formatted report matches the agreed template.


In [ ]:

trend = poisson_trend_test_plus(
    bin_summary,
    count='N',
    exposure='T',
    time_col='t_mid',
    alpha=0.05,
    se_method='robust',
    use_lrt=True,
    equivalence_rr=1.02,  # treat +/- 2 %/hour as practically equivalent
)
print(format_trend_report(trend, time_unit='fluence-bin', alpha=0.05))
trend



## Next actions

1. Replicate the binning for the remaining runs or larger extracts to cross-validate the sigma_SEE envelope.
2. Feed `bin_summary` into the Bayesian bathtub prototype (piecewise-constant hazards seeded by the Garwood intervals).
3. Integrate the trend-test result into the operations note, highlighting the statistical significance and practical-equivalence verdict.
